In [1]:
from langchain.agents import AgentExecutor, create_react_agent,initialize_agent,AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
import os

In [2]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
import os
import logging
from IPython.display import display, Markdown

In [3]:
# Set API Key
os.environ["GOOGLE_API_KEY"] = os.getenv("Gemini_API_KEY")

In [4]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [5]:
serper_api_key = os.getenv("SERPER_API_KEY")

In [6]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [7]:

# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0.7,
    convert_system_message_to_human=True
)

In [8]:
search = GoogleSerperAPIWrapper()

In [9]:
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

In [10]:
# Custom Medical Prompt Template
MEDICAL_PROMPT = PromptTemplate(
    input_variables=["input", "my_website", "competitors", "agent_scratchpad"],
    template="""You are a senior radiology AI analyst. Analyze and compare:

**Target Platform**: {my_website}
**Competitors**: {competitors}

Focus on:
1. FDA/CE certification status
2. DICOM/PACS integration capabilities
3. Supported imaging modalities (CT, MRI, X-ray)
4. Clinical validation documentation
5. Radiologist workflow optimization

Provide a detailed technical analysis in markdown format with comparison tables.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

{agent_scratchpad}"""
)

In [11]:


# Create agent with empty tools configuration
agent = create_react_agent(
    llm=llm,
    tools= tools ,  # Empty tools list
    prompt=MEDICAL_PROMPT.partial(
        tools="",  # Empty string for tools
        tool_names=""  # Empty string for tool names
    )
)


In [12]:
# Create executor
medical_agent = AgentExecutor(
    agent=agent,
    tools=[],  # Empty tools list
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)



In [13]:
# Run analysis
try:
    result = medical_agent.invoke({
        "input": "Medical imaging platform analysis",
        "my_website": "mlhealth360.com",
        "competitors": ["radai.com"]
    })
    print(result['output'])
except Exception as e:
    print(f"Analysis Error: {str(e)}")



> Entering new AgentExecutor chain...


c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: Analyze and compare mlhealth360.com and radai.com, focusing on FDA/CE certification status, DICOM/PACS integration capabilities, supported imaging modalities, clinical validation documentation, and radiologist workflow optimization.

Thought: I need to research both platforms to gather information on their features and capabilities.  Since direct access to internal documentation is likely limited, I'll rely on publicly available information like websites, publications, and news releases.  I will organize the information into a comparison table.

Action: [Intermediate Answer]
Action Input: Research mlhealth360.com and radai.com focusing on the five key areas.[Intermediate Answer] is not a valid tool, try one of [].

c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: Analyze and compare mlhealth360.com and radai.com, focusing on FDA/CE certification status, DICOM/PACS integration capabilities, supported imaging modalities, clinical validation documentation, and radiologist workflow optimization.

Thought: I need to research both platforms to gather information on their features and capabilities. Since direct access to internal documentation is likely limited, I'll rely on publicly available information like websites, publications, and news releases. I will organize the information into a comparison table.

Action: [Search]
Action Input: Search for information on mlhealth360.com and radai.com related to FDA/CE certifications, DICOM/PACS integration, supported modalities, clinical validation, and workflow optimization.[Search] is not a valid tool, try one of [].

c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: Analyze and compare mlhealth360.com and radai.com, focusing on FDA/CE certification status, DICOM/PACS integration capabilities, supported imaging modalities, clinical validation documentation, and radiologist workflow optimization.

Thought: I need to research both platforms to gather information on their features and capabilities. Since direct access to internal documentation is likely limited, I'll rely on publicly available information like websites, publications, and news releases. I will organize the information into a comparison table.  I'll also acknowledge the limitations of relying solely on publicly available data.

Action: [Research]
Action Input:  mlhealth360.com and radai.com focusing on FDA/CE certifications, DICOM/PACS integration, supported modalities, clinical validation, and workflow optimization.[Research] is not a valid tool, try one of [].

> Finished chain.
Agent stopped due to iteration limit or time limit.


In [14]:
print(type(result))  # Check if it's a dict, list, etc.

<class 'dict'>


In [15]:
from IPython.display import display
display(result)

{'input': 'Medical imaging platform analysis',
 'my_website': 'mlhealth360.com',
 'competitors': ['radai.com'],
 'output': 'Agent stopped due to iteration limit or time limit.'}